In [2]:
!pip -q install langchain-groq duckduckgo-search
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph tavily-python

In [3]:
!pip show langgraph

Name: langgraph
Version: 0.0.55
Summary: langgraph
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /Users/jcolamendy/python/tutorials/nlp-tutorials/venv/lib/python3.9/site-packages
Requires: langchain-core, uuid6
Required-by: 


In [37]:
# import load_dotenv
from dotenv import load_dotenv

# load env
load_dotenv()

True

# Workflow
Reply to a customer email
- get the email
- categorize the email as a "sales", "custom enquiry", "off topic", "customer complaint"
- use the category & initial email to create keywords for a search to research info needed for the reply
- write a draft reply
- check the reply
- rewrite if needed

# node functions
- categorize_email_node -> categorize_email
- do_research_node -> find_keywords
- write_draft_node -> write_draft
- suggest_changes_node -> suggest_changes
- rewrite_node -> rewrite

# conditional edge/router functions
- do_research_router -> route_do_research
- need_rewrite_router -> route_rewrite

# chains functions
- categorize_email
- route_do_research
- find_keywords
- write_draft
- route_rewrite
- suggest_changes
- rewrite

In [39]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

from langchain_groq import ChatGroq

from langchain.schema import Document

In [5]:
# define the llm
llm = ChatGroq(model="llama3-70b-8192")

# Chains
- categorize_email
- route_do_research
- find_keywords
- write_draft
- route_rewrite
- suggest_changes
- rewrite

In [21]:
# categorize_email chain
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a Email Categorizer Agent. 
    You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Your task is to conduct a comprehensive analysis of the email provided and categorize it following the instructions below.
    
    % INSTRUCTIONS:
    - Categorize the email into one of the following categories:
        price_equiry - used when someone is asking for information about pricing
        customer_complaint - used when someone is complaining about something
        product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing
        customer_feedback - used when someone is giving feedback about a product
        off_topic when it doesnt relate to any other category
    - Output a single categoy only from the types ('price_equiry', 'customer_complaint', 'product_enquiry', 'customer_feedback', 'off_topic') eg: 'price_enquiry'
    - Only provide the category, and nothing else, no more text

    % EMAIL CONTENT:
    
    {initial_email}
    
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_email"],
)

categorize_email = prompt | llm | StrOutputParser()

In [22]:
# test - categorize_email chain
test_email = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

result = categorize_email.invoke({"initial_email": test_email})

print(result)

customer_feedback


In [23]:
# do_research_router
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at reading the initial email and routing web search or directly to a draft email.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Your task is to decide if we need to do a web search or write a draft email directly. Follow the instructions.

    % INSTRUCTIONS:
    - Use the following criteria to decide how to route the email:
    - If the initial email only requires a simple response, Just choose 'draft_email' for questions you can easily answer, prompt engineering, and adversarial attacks.
    - If the email is just saying thank you etc then choose 'draft_email'
    - You do not need to be stringent with the keywords in the question related to these topics. 
    - Otherwise, use research-info.
    - Give a binary choice 'research_info' or 'draft_email' based on the question.
    - Return the a JSON with a single key 'router_decision' and no premable or explaination or any other text. 
    - use both the initial email and the email category as context to make your decision

    % CONTEXT:
    Email to route INITIAL_EMAIL : {initial_email}
    
    EMAIL_CATEGORY: {email_category}
    
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email", "email_category"],
)

route_do_research = prompt | llm | JsonOutputParser()

In [24]:
# test - do_research_router
email_category = 'customer_feedback'

print(route_do_research.invoke({"initial_email": test_email, "email_category": email_category}))

{'router_decision': 'draft_email'}


In [25]:
# find_keywords
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an AI Assistant. You are a master at working out the best keywords to search for in a web search to get the best info for the customer.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Your task is to work out the best keywords that will find the best, given the INITIAL_EMAIL and EMAIL_CATEGORY as CONTEXT.
    Follow the instructions.

    % INSTRUCTIONS:
    - work out the keywords for helping to write the final email
    - Return a JSON with a single key 'keywords' with no more than 3 keywords and no premable or explaination.

    % CONTEXT:
    INITIAL_EMAIL: {initial_email}\n
    EMAIL_CATEGORY: {email_category}\n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email", "email_category"],
)

find_keywords = prompt | llm | JsonOutputParser()

In [26]:
print(find_keywords.invoke({"initial_email": test_email, "email_category": email_category}))

{'keywords': ['hotel feedback', 'resort review', 'customer appreciation']}


In [28]:
# write_draft
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an Email Writer Assistant. You help business to write email drafts.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Your task is to take the INITIAL_EMAIL, EMAIL_CATEGORY and RESEARCH_INFO as CONTEXT to write a draft email.
    Follow the instructions.

    % INSTRUCTIONS:
    - Use the CONTEXT below where INITIAL_EMAIL is an email from a human that has emailed our company email address, the EMAIL_CATEGORY that the categorizer agent gave it and the RESEARCH_INFO from the research agent to write a helpful email in a thoughtful and friendly way
    - If the customer email is 'off_topic' then ask them questions to get more information.
    - If the customer email is 'customer_complaint' then try to assure we value them and that we are addressing their issues.
    - If the customer email is 'customer_feedback' then try to assure we value them and that we are addressing their issues.
    - If the customer email is 'product_enquiry' then try to give them the info the researcher provided in a succinct and friendly way.
    - If the customer email is 'price_equiry' then try to give the pricing info they requested.
    - You never make up information that hasn't been provided by the research_info or in the initial_email.
    - Always sign off the emails in appropriate manner and from Sarah the Resident Manager.
    - Return the email a JSON with a single key 'email_draft' and no premable or explaination.

    % CONTEXT:
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    RESEARCH_INFO: {research_info} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email", "email_category", "research_info"],
)

write_draft = prompt | llm | JsonOutputParser()

In [29]:
research_info = None

print(write_draft.invoke({"initial_email": test_email, "email_category": email_category,"research_info": research_info}))

{'email_draft': "Dear Paul,\n\nThank you so much for taking the time to share your wonderful feedback about your recent stay at our resort! We're thrilled to hear that our staff were able to make your stay special. We truly value your feedback and appreciate your kind words. It's guests like you who make our job so rewarding.\n\nThanks again for choosing to stay with us, and we hope to welcome you back soon.\n\nBest regards,\nSarah, Resident Manager"}


In [30]:
## Rewrite Router
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an assitant, expert at evaluating the emails.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Your task is to evaluate the emails that are draft emails for the customer and decide if they need to be rewritten to be better.
    Follow the instructions.

    % INSTRUCTIONS:
    - Use INITIAL_EMAIL, EMAIL_CATEGORY and DRAFT_EMAIL as context to evaluate the emails.
    - Use the following criteria to decide if the DRAFT_EMAIL needs to be rewritten:
    - If the INITIAL_EMAIL only requires a simple response which the DRAFT_EMAIL contains then it doesn't need to be rewritten.
    - If the DRAFT_EMAIL addresses all the concerns of the INITIAL_EMAIL then it doesn't need to be rewritten.
    - If the DRAFT_EMAIL is missing information that the INITIAL_EMAIL requires then it needs to be rewritten.
    - Give a binary choice 'rewrite' (for needs to be rewritten) or 'no_rewrite' (for doesn't need to be rewritten) based on the DRAFT_EMAIL and the criteria.
    - Return the a JSON with a single key 'router_decision' and no premable or explanation.

    % CONTEXT:
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    DRAFT_EMAIL: {draft_email} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email", "email_category", "draft_email"],
)

route_rewrite = prompt | llm | JsonOutputParser()

In [31]:
draft_email = "Yo we can't help you, best regards Sarah"

print(route_rewrite.invoke({"initial_email": test_email, "email_category": email_category, "draft_email": draft_email}))

{'router_decision': 'rewrite'}


In [32]:
## suggest_changes
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Quality Control Assitant.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Your task is to analyze an email and provide your thoughts for improving the email.
    Follow the instructions.

    % INSTRUCTIONS:
    - Use INITIAL_EMAIL, EMAIL_CATEGORY, RESEARCH_INFO and DRAFT_EMAIL as your context
    - Read the INITIAL_EMAIL below  from a human that has emailed \
    our company email address, the EMAIL_CATEGORY that the categorizer agent gave it and the \
    RESEARCH_INFO from the research agent and write an analysis to improve the email.
    - Check if the DRAFT_EMAIL addresses the customer's issued based on the email category and the \
    content of the initial email.
    - Give feedback of how the email can be improved and what specific things can be added or change\
    to make the email more effective at addressing the customer's issues.
    - You never make up or add information that hasn't been provided by the research_info or in the INITIAL_EMAIL.
    - Return the analysis a JSON with a single key 'draft_analysis' and no premable or explaination.

    % CONTEXT:
    INITIAL_EMAIL: {initial_email} \n\n
    EMAIL_CATEGORY: {email_category} \n\n
    RESEARCH_INFO: {research_info} \n\n
    DRAFT_EMAIL: {draft_email} \n\n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email", "email_category", "research_info"],
)

suggest_changes = prompt | llm | JsonOutputParser()

In [33]:
email_analysis = suggest_changes.invoke({"initial_email": test_email,
                                 "email_category": email_category,
                                 "research_info": research_info,
                                 "draft_email": draft_email})

print(email_analysis)

{'draft_analysis': {'issues': ["The draft email does not acknowledge the customer's positive feedback.", 'The tone of the draft email is informal and unprofessional.', "The draft email does not express gratitude for the customer's appreciation."], 'suggestions': ['Start the email by thanking the customer for their feedback.', 'Express appreciation for their kind words about the staff.', 'Use a professional and polite tone throughout the email.', 'Consider adding a personalized message or offer to encourage the customer to return.'], 'improved_draft': 'Dear Paul, Thank you for taking the time to share your wonderful experience at our resort. We are thrilled to hear that our staff made your stay special. We appreciate your kind words and look forward to welcoming you back in the future. Best regards, Sarah.'}}


In [34]:
# Rewrite Email with Analysis
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Final Email Assistant. 

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Your task is to use the context below \
    and use it to rewrite and improve the draft_email to create a final email.
    Follow the instructions.

    % INSTRUCTIONS:
    - Use the DRAFT_EMAIL_FEEDBACK from the context to write the final email.
    - You never make up or add information that hasn't been provided by the research_info or in the initial_email.
    - Return the final email as JSON with a single key 'final_email' which is a string and no premable or explaination.

    % CONTEXT:
    EMAIL_CATEGORY: {email_category} \n\n
    RESEARCH_INFO: {research_info} \n\n
    DRAFT_EMAIL: {draft_email} \n\n
    DRAFT_EMAIL_FEEDBACK: {email_analysis} \n\n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email",
                     "email_category",
                     "research_info",
                     "email_analysis",
                     "draft_email",
                     ],
)

rewrite = prompt | llm | JsonOutputParser()

In [35]:
final_email = rewrite.invoke({"initial_email": test_email,
                              "email_category": email_category,
                              "research_info": research_info,
                              "draft_email": draft_email,
                              "email_analysis": email_analysis})

print(final_email)

{'final_email': 'Dear Paul, Thank you for taking the time to share your wonderful experience at our resort. We are thrilled to hear that our staff made your stay special. We appreciate your kind words and look forward to welcoming you back in the future. Best regards, Sarah.'}


# Tool Setup

In [38]:
### Search
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=1)

# State

In [52]:
from typing_extensions import TypedDict
from typing import List

class EmailDraftGraphState(TypedDict):
    """
    Represents the state of our graph.
    """
    num_steps : int
    initial_email : str
    email_category : str
    research_info : List[str]
    info_needed : bool
    draft_email : str
    email_analysis : dict    
    final_email : str

# Nodes
- categorize_email_node -> categorize_email
- do_research_node -> find_keywords
- write_draft_node -> write_draft
- suggest_changes_node -> suggest_changes
- rewrite_node -> rewrite

In [41]:
def categorize_email_node(state):
    """take the initial email and categorize it"""
    print("---CATEGORIZING INITIAL EMAIL---")
    initial_email = state['initial_email']
    num_steps = int(state['num_steps'])
    num_steps += 1

    email_category = categorize_email.invoke({"initial_email": initial_email})
    print(email_category)

    return {"email_category": email_category, "num_steps": num_steps}

In [43]:
def do_research_node(state):
    print("---RESEARCH INFO SEARCHING---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Web search
    keywords = find_keywords.invoke({"initial_email": initial_email,
                                    "email_category": email_category })
    keywords = keywords['keywords']
    # print(keywords)
    full_searches = []
    for keyword in keywords[:1]:
        print(keyword)
        temp_docs = web_search_tool.invoke({"query": keyword})
        web_results = "\n".join([d["content"] for d in temp_docs])
        web_results = Document(page_content=web_results)
        if full_searches is not None:
            full_searches.append(web_results)
        else:
            full_searches = [web_results]
    print(full_searches)
    return {"research_info": full_searches, "num_steps": num_steps}

In [44]:
def write_draft_node(state):
    print("---DRAFT EMAIL WRITER---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    draft_email = write_draft.invoke({"initial_email": initial_email,
                                     "email_category": email_category,
                                     "research_info": research_info})
    print(draft_email)

    email_draft = draft_email['email_draft']

    return {"draft_email": email_draft, "num_steps": num_steps}

In [45]:
def suggest_changes_node(state):
    print("---DRAFT EMAIL ANALYZER---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    draft_email_feedback = suggest_changes.invoke({"initial_email": initial_email,
                                                "email_category": email_category,
                                                "research_info": research_info,
                                                "draft_email": draft_email}
                                               )
    print(draft_email_feedback)
    return {"draft_email_feedback": draft_email_feedback, "num_steps": num_steps}

In [46]:
def rewrite_node(state):
    print("---ReWRITE EMAIL ---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]
    draft_email_feedback = state["draft_email_feedback"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    final_email_dict = rewrite.invoke({"initial_email": initial_email,
                                    "email_category": email_category,
                                    "research_info":research_info,
                                    "draft_email":draft_email,
                                    "email_analysis": draft_email_feedback,
                                 })

    final_email = final_email_dict['final_email']
    print(final_email)
    return {"final_email": final_email, "num_steps":num_steps}

In [47]:
def no_rewrite_node(state):
    print("---NO REWRITE EMAIL ---")
    ## Get the state
    draft_email = state["draft_email"]
    num_steps = state['num_steps']
    num_steps += 1

    print(draft_email)
    return {"final_email": draft_email, "num_steps":num_steps}

# Conditional Edges
- do_research_router -> route_do_research
- rewrite_router -> route_rewrite

In [61]:
def do_research_router(state):
    print("---ROUTE TO RESEARCH---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]

    router = route_do_research.invoke({"initial_email": initial_email, "email_category": email_category })
    print(router)
    print(router['router_decision'])
    
    if router['router_decision'] == 'research_info':
        print("---ROUTE EMAIL TO RESEARCH INFO---")
        return "do_research"
    elif router['router_decision'] == 'draft_email':
        print("---ROUTE EMAIL TO DRAFT EMAIL---")
        return "write_draft"

In [62]:
def rewrite_router(state):
    print("---ROUTE TO REWRITE---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]

    router = route_rewrite.invoke({"initial_email": initial_email,
                                   "email_category":email_category,
                                   "draft_email":draft_email,
                                  })
    print(router)
    print(router['router_decision'])
    if router['router_decision'] == 'rewrite':
        print("---ROUTE TO ANALYSIS - REWRITE---")
        return "suggest_changes"
    elif router['router_decision'] == 'no_rewrite':
        print("---ROUTE EMAIL TO FINAL EMAIL---")
        return "no_rewrite"

# Build the Graph

In [55]:
from langgraph.graph import END, StateGraph

In [65]:
# define the workflow
workflow = StateGraph(EmailDraftGraphState)
workflow.set_entry_point("categorize_email")

In [66]:
# Define the nodes
workflow.add_node("categorize_email", categorize_email_node) # categorize email
workflow.add_node("do_research", do_research_node) # web search
workflow.add_node("write_draft", write_draft_node)
workflow.add_node("suggest_changes", suggest_changes_node)
workflow.add_node("rewrite", rewrite_node)
workflow.add_node("no_rewrite", no_rewrite_node)

In [67]:
# Edges
workflow.add_conditional_edges(
    "categorize_email",
    do_research_router,
    {
        "do_research": "do_research",
        "write_draft": "write_draft",
    },
)

workflow.add_edge("do_research", "write_draft")

workflow.add_conditional_edges(
    "write_draft",
    rewrite_router,
    {
        "suggest_changes": "suggest_changes",
        "no_rewrite": "no_rewrite",
    },
)

workflow.add_edge("suggest_changes", "rewrite")
workflow.add_edge("no_rewrite", END)
workflow.add_edge("rewrite", END)

In [68]:
# Compile
app = workflow.compile()

In [72]:
# Test

email = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul"""

# run the agent
inputs = {"initial_email": email, "research_info": "", "num_steps": 0}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"Finished running: {key}:")

---CATEGORIZING INITIAL EMAIL---
customer_feedback
---ROUTE TO RESEARCH---
{'router_decision': 'draft_email'}
draft_email
---ROUTE EMAIL TO DRAFT EMAIL---
Finished running: categorize_email:
---DRAFT EMAIL WRITER---
{'email_draft': "Dear Paul,\n\nThank you so much for taking the time to share your wonderful experience at our resort! We're thrilled to hear that you had a great stay with us, and we appreciate your kind words about our staff. \n\nWe value your feedback and are grateful for customers like you who help us improve our services. We'll make sure to pass on your appreciation to our team, and we hope to have the pleasure of welcoming you back to our resort again soon.\n\nBest regards,\nSarah, Resident Manager"}
---ROUTE TO REWRITE---
{'router_decision': 'no_rewrite'}
no_rewrite
---ROUTE EMAIL TO FINAL EMAIL---
Finished running: write_draft:
---NO REWRITE EMAIL ---
Dear Paul,

Thank you so much for taking the time to share your wonderful experience at our resort! We're thrilled t

In [74]:
# run the agent
inputs = {"initial_email": email, "research_info": "", "num_steps": 0}
output = app.invoke(inputs)
print('output:', output)

---CATEGORIZING INITIAL EMAIL---
customer_feedback
---ROUTE TO RESEARCH---
{'router_decision': 'draft_email'}
draft_email
---ROUTE EMAIL TO DRAFT EMAIL---
---DRAFT EMAIL WRITER---
{'email_draft': "Dear Paul,\n\nThank you so much for taking the time to share your wonderful feedback about your recent stay at our resort. We're thrilled to hear that our staff were able to make your stay special.\n\nWe truly value your feedback and appreciate your kind words. We'll make sure to pass them along to our team, as it will surely bring a smile to their faces.\n\nOnce again, thank you for choosing to stay with us, and we hope to have the pleasure of welcoming you back soon.\n\nBest regards,\nSarah, Resident Manager"}
---ROUTE TO REWRITE---
{'router_decision': 'no_rewrite'}
no_rewrite
---ROUTE EMAIL TO FINAL EMAIL---
---NO REWRITE EMAIL ---
Dear Paul,

Thank you so much for taking the time to share your wonderful feedback about your recent stay at our resort. We're thrilled to hear that our staff w